<a href="https://colab.research.google.com/github/sajib-222/Data-Mining-and-warehouse/blob/main/Assignment(5)_My_Search_engine_ID_2125051016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Shariful Islam Sajib Sarker
## ID: 2125051016

In [27]:
import networkx as nx
import numpy as np
import pandas as pd

In [28]:
import requests
from bs4 import BeautifulSoup

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

STOPWORDS = stopwords.words('english')
print(STOPWORDS)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
custom_STOPWORDS = [] # Add your own stopwords here
STOPWORDS.extend(custom_STOPWORDS)

In [31]:
from collections import defaultdict

# Inverted index: word -> set of URLs
inverted_index = defaultdict(set)
url_list = set()

In [32]:
# This dictionary will be used to build the connection between links
web_connection = {'source':[], 'target':[]}

In [33]:
import re

# This function will clean the content of web page in order to build the inverted index.
def clean_and_tokenize(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())  # Remove punctuation and lowercase
    tokens = text.split()
    return [t for t in tokens if t not in STOPWORDS and len(t) > 1]

In [34]:
from urllib.parse import urljoin, urlparse

# The crawl function has 5 parameters
# url = The url to crawl
# base_domain = the base domain of the url. During crawling, the crawler will ignore links from other domains

def crawl(url, base_domain, visited, visit_limit, limit):
    if limit==0 or len(visited)==visit_limit:
        return

    try:
        response = requests.get(url, timeout=5)
        if response.status_code != 200:
            return
    except requests.RequestException:
        return

    visited.add(url)
    print("-"*(10-limit), end=" ")
    print(f"Crawled: {url}")

    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text(separator=' ', strip=True)
    words = clean_and_tokenize(text)

    for word in words:
        inverted_index[word].add(url)
        url_list.add(url)

    # Recursively follow links
    for tag in soup.find_all('a', href=True):
        link = urljoin(url, tag['href'])
        parsed = urlparse(link)

        # Store external links as connection
        web_connection['source'].append(url)
        web_connection['target'].append(link)

        if parsed.netloc == base_domain and link not in visited:
            crawl(link, base_domain, visited, visit_limit, limit-1)

In [35]:
def crawl_roots(root_urls, max_per_root=2, visit_limit=50):
    for root in root_urls:
        print(f"\nStarting crawl from: {root}")
        domain = urlparse(root).netloc
        visited = set()
        crawl(root, domain, visited, visit_limit, max_per_root)

In [36]:
seed_urls = [
    'https://www.bbc.com/news',
    'https://www.theguardian.com/world',
]

crawl_roots(seed_urls, max_per_root=10)


Starting crawl from: https://www.bbc.com/news
 Crawled: https://www.bbc.com/news
- Crawled: https://www.bbc.com/news#main-content
-- Crawled: https://www.bbc.com/
--- Crawled: https://www.bbc.com/#main-content
---- Crawled: https://www.bbc.com/sport
----- Crawled: https://www.bbc.com
------ Crawled: https://www.bbc.com#main-content
------- Crawled: https://www.bbc.com/business
-------- Crawled: https://www.bbc.com/business#main-content
--------- Crawled: https://www.bbc.com/innovation
--------- Crawled: https://www.bbc.com/culture
--------- Crawled: https://www.bbc.com/arts
--------- Crawled: https://www.bbc.com/travel
--------- Crawled: https://www.bbc.com/future-planet
--------- Crawled: https://www.bbc.com/audio
--------- Crawled: https://www.bbc.com/video
--------- Crawled: https://www.bbc.com/live
--------- Crawled: https://www.bbc.com/business/executive-lounge
--------- Crawled: https://www.bbc.com/business/technology-of-business
--------- Crawled: https://www.bbc.com/business/f

In [37]:
# print inverted index
print("\nSample inverted index (first 20 words):")
for word in list(inverted_index.keys())[:20]:
    print(f"{word}: {list(inverted_index[word])}")


Sample inverted index (first 20 words):
home: ['https://www.theguardian.com/world/ukraine', 'https://www.bbc.com/news#main-content', 'https://www.theguardian.com/world', 'https://www.theguardian.com/sport/tennis', 'https://www.theguardian.com/preference/edition/uk', 'https://www.theguardian.com/sport/rugby-union', 'https://www.theguardian.com/preference/edition/int#navigation', 'https://www.theguardian.com/culture', 'https://www.theguardian.com/business', 'https://www.theguardian.com/profile/editorial', 'https://www.theguardian.com/environment', 'https://www.theguardian.com/preference/edition/us', 'https://www.theguardian.com/sport/boxing', 'https://www.bbc.com', 'https://www.bbc.com/news/scotland/scotland_politics', 'https://www.bbc.com/news/world/australia', 'https://www.theguardian.com/tone/cartoons', 'https://www.theguardian.com/sport/us-sport', 'https://www.bbc.com/news/world/middle_east', 'https://www.bbc.com/innovation', 'https://www.theguardian.com/', 'https://www.bbc.com/#mai

In [38]:
# Print first 20 connections

for source, target in list(zip(web_connection['source'], web_connection['target']))[:20]:
    print(f"{source} -> {target}")

https://www.bbc.com/news -> https://www.bbc.com/news#main-content
https://www.bbc.com/news#main-content -> https://www.bbc.com/news#main-content
https://www.bbc.com/news#main-content -> https://www.bbc.com/
https://www.bbc.com/ -> https://www.bbc.com/#main-content
https://www.bbc.com/#main-content -> https://www.bbc.com/#main-content
https://www.bbc.com/#main-content -> https://www.bbc.com/
https://www.bbc.com/#main-content -> https://www.bbc.com/
https://www.bbc.com/#main-content -> https://www.bbc.com/news
https://www.bbc.com/#main-content -> https://www.bbc.com/sport
https://www.bbc.com/sport -> https://www.bbc.com
https://www.bbc.com -> https://www.bbc.com#main-content
https://www.bbc.com#main-content -> https://www.bbc.com#main-content
https://www.bbc.com#main-content -> https://www.bbc.com/
https://www.bbc.com#main-content -> https://www.bbc.com/
https://www.bbc.com#main-content -> https://www.bbc.com/news
https://www.bbc.com#main-content -> https://www.bbc.com/sport
https://www.

In [54]:
web_graph = nx.DiGraph()
for source, target in zip(web_connection['source'], web_connection['target']):
  web_graph.add_edge(source, target) # Iterate through source and target lists using zip

In [55]:
pagerank_scores = nx.pagerank(web_graph, alpha=0.85, max_iter=100, tol=1e-6)
print("\nPageRank Scores:", pagerank_scores)


PageRank Scores: {'https://www.bbc.com/news': 0.0003793044975203973, 'https://www.bbc.com/news#main-content': 0.00026070973835921033, 'https://www.bbc.com/': 0.00037496161849412765, 'https://www.bbc.com/#main-content': 0.000259470191973223, 'https://www.bbc.com/sport': 0.0003793044975203973, 'https://www.bbc.com': 0.000260847777381605, 'https://www.bbc.com#main-content': 0.0002589614217905827, 'https://www.bbc.com/business': 0.0003769151094759742, 'https://www.bbc.com/business#main-content': 0.0002602527647746279, 'https://www.bbc.com/innovation': 0.0003769151094759742, 'https://www.bbc.com/innovation#main-content': 0.00025881785805791904, 'https://www.bbc.com/culture': 0.0003793044975203973, 'https://www.bbc.com/arts': 0.00037496161849412765, 'https://www.bbc.com/travel': 0.0003793044975203973, 'https://www.bbc.com/future-planet': 0.0003769151094759742, 'https://www.bbc.com/audio': 0.0003769151094759742, 'https://www.bbc.com/video': 0.0003769151094759742, 'https://www.bbc.com/live': 

In [58]:
def search_engine(query, index, scores):
    query_terms = query.lower().split()
    results = set()
    for term in query_terms:
        if term in index:
            if not results:
                results = set(index[term])
            else:
                results = results.intersection(index[term])  # Find common websites

    # Sort results based on score
    ranked_results = []
    for website in results:
        if website in scores:
          ranked_results.append((website, scores[website]))
    ranked_results.sort(key=lambda x: x[1], reverse=True)

    return ranked_results

In [60]:
# Query and display results
query = "neural network"
print(f"\nSearch Results for '{query}' using PageRank:")
results = search_engine(query, inverted_index, pagerank_scores)

for page, score in results:
    print(f"{page}: ({score})")




Search Results for 'neural network' using PageRank:
https://www.bbc.com/travel: (0.0003793044975203973)
https://www.bbc.com/video: (0.0003769151094759742)
https://www.theguardian.com/email-newsletters: (0.0003388247264083948)
https://www.theguardian.com/preference/edition/au: (0.0003388247264083948)
https://www.theguardian.com/sport/cricket: (0.0003376510509324923)
https://www.theguardian.com/sport/rugbyleague: (0.0002642176921745373)
